## Setup

In [ ]:
import sys
sys.path.insert(0, '../src')

import vibe_widget as vw
import pandas as pd
import numpy as np
import os

API_KEY = os.getenv("ANTHROPIC_API_KEY")

if not API_KEY:
    print("⚠️ Please set ANTHROPIC_API_KEY environment variable")
else:
    print("✅ API key loaded")

## Example 1: Linked Scatter Plot & Histogram

Let's create two widgets that communicate:
1. **Scatter plot** with brush selection (exports `selected_indices`)
2. **Histogram** that highlights selected points (imports `selected_indices`)

When you brush-select points in the scatter plot, the histogram automatically updates!

In [ ]:
# Create sample dataset
np.random.seed(42)

df = pd.DataFrame({
    'temperature': np.random.normal(20, 5, 200),
    'humidity': np.random.uniform(30, 90, 200),
    'sensor_id': np.random.choice(['A', 'B', 'C', 'D'], 200),
})

print(f"📊 Created dataset with {len(df)} data points")
df.head()

In [ ]:
# Widget 1: Scatter plot with brush selection
scatter = vw.create(
    "scatter plot of temperature vs humidity with brush selection, color by sensor_id",
    df,
    api_key=API_KEY,
    show_progress=True,
    exports={
        "selected_indices": "array of indices of selected points from brush selection"
    }
)

print("\n✨ Scatter plot created with brush selection!")
print("🖱️ Try brushing over points to select them")

In [ ]:
# Widget 2: Histogram that imports selection from scatter plot
histogram = vw.create(
    "histogram of temperature with two colors: selected points in orange, unselected in gray",
    df,
    api_key=API_KEY,
    show_progress=True,
    imports={
        "selected_indices": scatter
    }
)

print("\n✨ Histogram created and linked to scatter plot!")
print("🔗 Selected points in scatter → Highlighted in histogram")

### How It Works

```python
# Widget A exports a trait
scatter = vw.create(
    ...,
    exports={"selected_indices": "description"}
)

# Widget B imports that trait
histogram = vw.create(
    ...,
    imports={"selected_indices": scatter}
)
```

Vibe Widget automatically:
1. ✅ Creates the trait on the exporting widget
2. ✅ Links it bidirectionally using traitlets
3. ✅ Updates all importing widgets when the trait changes
4. ✅ Generates code that listens for trait changes

## Example 2: 2D Terrain Painter → 3D Landscape

This is one of the coolest examples! We'll create:
1. **2D Canvas**: Paint terrain height with your mouse (exports `heightmap`)
2. **3D Viewer**: Renders the terrain in 3D using Three.js (imports `heightmap`)

Paint on the canvas and watch it render in 3D in real-time!

In [ ]:
# Widget 1: 2D terrain painter
painter = vw.create(
    """2D canvas for painting terrain height with mouse brush.
    - Click and drag to paint elevation
    - Store heightmap as 64x64 grid of floats 0-1
    - Use high intensity brushes for dramatic terrain
    - Show a gradient from blue (low) to white (high)
    """,
    df=None,  # No initial data needed
    api_key=API_KEY,
    show_progress=True,
    exports={
        "heightmap": "64x64 grid of float values (0-1) representing terrain elevation"
    }
)

print("\n🎨 2D Terrain painter created!")
print("🖱️ Click and drag to paint terrain")

In [ ]:
# Widget 2: 3D landscape viewer
landscape = vw.create(
    """3D landscape viewer using Three.js (import via ESM).
    - Create terrain mesh from heightmap (64x64 grid)
    - Use PlaneGeometry and displace vertices based on heightmap values
    - Add orbit controls for rotation and zoom
    - Use gradient coloring: blue (low) → green (mid) → white (high)
    - Add a water plane at elevation 0.2
    - Update mesh when heightmap changes
    """,
    df=None,
    api_key=API_KEY,
    show_progress=True,
    imports={
        "heightmap": painter
    }
)

print("\n🏔️ 3D Landscape viewer created and linked!")
print("🔗 Paint on canvas → See in 3D instantly")

### Bonus: Simulate Rain Erosion

Let's add a button that simulates hydraulic erosion on the terrain!

In [ ]:
import ipywidgets as widgets
from IPython.display import display

GRID_SIZE = 64

def simulate_erosion(b):
    """Simulate hydraulic erosion on the terrain"""
    btn.description = "🌧️ Raining..."
    btn.disabled = True
    
    # Get current heightmap
    h_list = painter.heightmap or [0.0] * (GRID_SIZE * GRID_SIZE)
    grid = np.array(h_list).reshape((GRID_SIZE, GRID_SIZE))
    
    # Simulation parameters
    drops = 2000
    erosion_rate = 0.01
    deposition_rate = 0.005
    
    # Simple hydraulic erosion
    for _ in range(drops):
        # Random raindrop start position
        x, y = np.random.randint(0, GRID_SIZE, 2)
        
        path_len = 0
        while path_len < 30:  # Max path length
            path_len += 1
            
            # Find lowest neighbor
            best_nx, best_ny = x, y
            min_h = grid[y, x]
            
            for dy in [-1, 0, 1]:
                for dx in [-1, 0, 1]:
                    if dx == 0 and dy == 0:
                        continue
                    nx, ny = x + dx, y + dy
                    if 0 <= nx < GRID_SIZE and 0 <= ny < GRID_SIZE:
                        if grid[ny, nx] < min_h:
                            min_h = grid[ny, nx]
                            best_nx, best_ny = nx, ny
            
            if best_nx == x and best_ny == y:
                # Local minimum (pool) - deposit sediment
                grid[y, x] += deposition_rate
                break
            else:
                # Flow downhill - erode current position
                grid[y, x] -= erosion_rate
                x, y = best_nx, best_ny
    
    # Clip and update widget
    grid = np.clip(grid, 0.0, 1.0)
    painter.heightmap = grid.flatten().tolist()
    
    btn.description = "🌧️ Simulate Rain Erosion"
    btn.disabled = False

btn = widgets.Button(
    description="🌧️ Simulate Rain Erosion",
    layout=widgets.Layout(width='100%', height='50px'),
    style={'button_color': '#4488ff'}
)
btn.on_click(simulate_erosion)

display(widgets.VBox([
    widgets.Label("Simulate 2000 raindrops eroding the terrain:"),
    btn
]))

## Example 3: Solar System Explorer

Create an interactive 3D solar system where clicking a planet highlights its data in a chart!

1. **3D Solar System**: Click planets to select them (exports `selected_planet`)
2. **Bar Chart**: Shows planet data with selection highlighting (imports `selected_planet`)

In [ ]:
# Load planets data
df_planets = pd.read_csv('../testdata/planets.csv')
print(f"🪐 Loaded {len(df_planets)} planets")
df_planets.head()

In [ ]:
# Widget 1: 3D Solar System
solar_system = vw.create(
    """3D solar system using Three.js showing planets orbiting the sun.
    - Create spheres for each planet with relative sizes
    - Position planets at their relative distances from sun
    - Make planets clickable to select them
    - Highlight selected planet with a bright glow/outline
    - Add orbit controls for rotation
    - Default selection: Earth
    - Export the selected planet name
    """,
    df=df_planets,
    api_key=API_KEY,
    show_progress=True,
    exports={
        "selected_planet": "name of the currently selected planet"
    }
)

print("\n🪐 3D Solar system created!")
print("🖱️ Click planets to select them")

In [ ]:
# Monitor selection changes
import ipywidgets as widgets
from IPython.display import display

label = widgets.Label(value=f"Selected: {solar_system.selected_planet or 'None'}")

def on_planet_change(change):
    label.value = f"Selected: {change.new or 'None'}"

solar_system.observe(on_planet_change, names='selected_planet')
display(label)

In [ ]:
# Widget 2: Planet data chart
planet_chart = vw.create(
    """Bar chart showing planet properties.
    - Add dropdown to switch between metrics: distance, mass, radius, orbital_period
    - Highlight the selected planet bar with an orange border and background
    - Sort bars by the selected metric
    - Show values on bars
    - Use a clean, modern design
    """,
    df=df_planets,
    api_key=API_KEY,
    show_progress=True,
    imports={
        "selected_planet": solar_system
    }
)

print("\n📊 Planet chart created and linked!")
print("🔗 Click planets in 3D → See highlighted in chart")

## Understanding Exports & Imports

### Exports

When you **export** a trait, you're saying: "This widget will expose this piece of state."

```python
widget = vw.create(
    "widget description",
    data,
    exports={
        "trait_name": "description of what this trait contains"
    }
)
```

The AI uses your description to:
- ✅ Generate code that updates this trait
- ✅ Choose the right data type (array, string, object, etc.)
- ✅ Set up event listeners to keep it synchronized

### Imports

When you **import** a trait, you're saying: "This widget needs to react to changes in another widget."

```python
widget2 = vw.create(
    "widget description",
    data,
    imports={
        "trait_name": source_widget
    }
)
```

The AI generates code that:
- ✅ Listens for changes to the imported trait
- ✅ Updates the visualization when the trait changes
- ✅ Handles the trait data appropriately

### Bidirectional Linking

Vibe Widget uses **traitlets** under the hood to create bidirectional links:

```
Widget A (exports)  ←→  Widget B (imports)
     ↓ trait change         ↑ automatically updated
```

This means changes flow automatically in both directions!

## Advanced Example: Tic-Tac-Toe with AI

For a more complex example of cross-widget communication, check out the full Tic-Tac-Toe demo in `test_cross_widget.ipynb`:

- 🎮 Interactive game board (exports `board_state`, `game_over`, `current_turn`)
- 🤖 AI move calculator (imports `board_state`, exports `ai_move`)
- 🌳 Decision tree visualizer (imports `board_state`)

Three widgets working together with complex state synchronization!

## Summary

### What We Learned:

✨ **Exports**: Let widgets share their state with others  
✨ **Imports**: Let widgets react to state changes from others  
✨ **Automatic Linking**: Vibe Widget handles all synchronization  
✨ **AI-Generated Code**: The LLM writes the glue code for you  

### Use Cases:

📊 **Linked Charts**: Filter, highlight, and coordinate multiple visualizations  
🎨 **Interactive Tools**: Create → View pipelines (2D → 3D, Edit → Preview)  
🎮 **Simulations**: Game state → AI controller → Visualizations  
🔍 **Exploratory Analysis**: Selection in one view → Details in another  

### The Magic:

You just describe what you want in natural language, and Vibe Widget:
1. Generates the export/import traits
2. Creates event listeners and update logic
3. Handles data serialization/deserialization
4. Links everything together automatically

**No manual state management. No callback hell. Just pure magic.** ✨